In [ ]:
sequence = "bus"

In [ ]:
!../sequences/$sequence/runme.sh

## Clipping -> Normalizing

In [ ]:
!sed -z -i 's!    out = clip(YUV.to_RGB(E_k) + 128)!    #out = clip(YUV.to_RGB(E_k) + 128)!g' image_IPP_adaptive.py
!sed -z -i 's!    dq_E_k = YUV.from_RGB(frame.read(prefix, k) - 128)\*1!    #dq_E_k = YUV.from_RGB(frame.read(prefix, k) - 128)\*1!g' image_IPP_adaptive.py
!sed -z -i 's!    #out, max, min = values.norm(YUV.to_RGB(E_k)); out \*= 255!    out, max, min = values.norm(YUV.to_RGB(E_k)); out \*= 255!g' image_IPP_adaptive.py
!sed -z -i 's!    #dq_E_k = YUV.from_RGB(values.denorm(frame.read(prefix, k)/255, max, min))!    dq_E_k = YUV.from_RGB(values.denorm(frame.read(prefix, k)/255, max, min))!g' image_IPP_adaptive.py

# Quantizing VS clipping

In [ ]:
!cat -n image_IPP_adaptive.py | grep "out, " 
!cat -n image_IPP_adaptive.py | grep "dq_E_k = YUV" 

In [ ]:
!python RD_curve.py 1> >(tee /tmp/output >&1) | grep BPP | grep Average | cat -n

In [ ]:
!grep BPP /tmp/output | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/normalizing.txt

In [ ]:
!cat /tmp/normalizing.txt

In [ ]:
RD_normalizing = []
with open("/tmp/normalizing.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_normalizing.append((float(rate), float(_distortion)))

## Normalizing -> Clipping

In [ ]:
!sed -z -i 's!    #out = clip(YUV.to_RGB(E_k) + 128)!    out = clip(YUV.to_RGB(E_k) + 128)!g' image_IPP_adaptive.py
!sed -z -i 's!    #dq_E_k = YUV.from_RGB(frame.read(prefix, k) - 128)\*1!    dq_E_k = YUV.from_RGB(frame.read(prefix, k) - 128)\*1!g' image_IPP_adaptive.py
!sed -z -i 's!    out, max, min = values.norm(YUV.to_RGB(E_k)); out \*= 255!    #out, max, min = values.norm(YUV.to_RGB(E_k)); out \*= 255!g' image_IPP_adaptive.py
!sed -z -i 's!    dq_E_k = YUV.from_RGB(values.denorm(frame.read(prefix, k)/255, max, min))!    #dq_E_k = YUV.from_RGB(values.denorm(frame.read(prefix, k)/255, max, min))!g' image_IPP_adaptive.py

In [ ]:
!cat -n image_IPP_adaptive.py | grep "out = " 
!cat -n image_IPP_adaptive.py | grep "dq_E_k = YUV" 

In [ ]:
!python RD_curve.py 1> >(tee /tmp/output >&1) | grep BPP | grep Average | cat -n

In [ ]:
!grep BPP /tmp/output | grep Average | cut -d ' ' -f 4,9 | tr ' ' '\t'  > /tmp/clipping.txt

In [ ]:
!cat /tmp/clipping.txt

In [ ]:
RD_clipping = []
with open("/tmp/clipping.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        RD_clipping.append((float(rate), float(_distortion)))

## Comparison

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.axes as ax
import pylab

pylab.figure(dpi=150)
pylab.plot(*zip(*RD_normalizing), label="Normalizing")
pylab.plot(*zip(*RD_clipping), label="Clipping")
pylab.title(f"{sequence}")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("AMSE")
plt.legend(loc='upper right')
pylab.show()